In [ ]:
import sys

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

sys.path.append(r'C:\Users\19167\AppData\Local\Programs\Python\smartone_quant\app')

In [ ]:
# 获取股票行情数据
"""
"time"                #时间戳
"open"                #开盘价
"high"                #最高价
"low"                 #最低价
"close"               #收盘价
"volume"              #成交量
"amount"              #成交额
"settle"              #今结算
"openInterest"        #持仓量
"""

In [ ]:
# 日期工具
from datetime import datetime, timedelta
from xtquant import xtdata

from app.utils.qmt_data_utils import parse_stock_data

In [ ]:
# 测试下载000001.SZ的日 k 行情数据并获取打印
stock_code: str = '000001.SZ'
print(f'开始下载股票{stock_code}的行情数据')
# 获取一年前的日期,yyyyMMdd格式
one_years_ago = (datetime.now() - timedelta(days=1 * 365)).strftime('%Y%m%d')
print(f'一年前的日期为：{one_years_ago}')
# 获取两年前的日期,yyyyMMdd格式
two_years_ago = (datetime.now() - timedelta(days=2 * 365)).strftime('%Y%m%d')
print(f'两年前的日期为：{two_years_ago}')
# 获取三年前的日期,yyyyMMdd格式
three_years_ago = (datetime.now() - timedelta(days=3 * 365)).strftime('%Y%m%d')
print(f'三年前的日期为：{three_years_ago}')
# 获取今天日期
today = datetime.now().strftime('%Y%m%d')
print(f'今天的日期为：{today}')

In [ ]:
# 获取市场数据
market_data = xtdata.get_market_data(
    field_list=["time", "open", "high", "low", "close", "volume", "amount"],
    stock_list=["000001.SZ"],
    period='1w',
    start_time=three_years_ago,
    end_time=today,
    count=-1,
    dividend_type='front',
    fill_data=False
)

if not market_data:
    print(f'没有获取到股票{stock_code}的行情数据')

print(market_data["time"])

In [38]:
from app.models.qmt_stock_weekly import QmtStockWeeklyOri

# 解析股票数据
stock_objs = parse_stock_data(market_data, model_cls=QmtStockWeeklyOri)
stock_objs

[2025-06-26 14:38:16,875] [INFO] 正在解析股票数据: 000001.SZ
[2025-06-26 14:38:16,946] [INFO] 解析完成，共解析出 152 条股票数据


[QmtStockWeeklyOri(stock_code='000001.SZ', time=datetime.datetime(2022, 7, 3, 0, 0), open=np.float64(12.4), high=np.float64(13.23), low=np.float64(12.38), close=np.float64(13.08), volume=6216923, amount=9139037824.0, id=None),
 QmtStockWeeklyOri(stock_code='000001.SZ', time=datetime.datetime(2022, 7, 10, 0, 0), open=np.float64(13.11), high=np.float64(13.43), low=np.float64(12.53), close=np.float64(12.7), volume=4363334, amount=6422741835.0, id=None),
 QmtStockWeeklyOri(stock_code='000001.SZ', time=datetime.datetime(2022, 7, 17, 0, 0), open=np.float64(12.66), high=np.float64(12.89), low=np.float64(11.38), close=np.float64(11.4), volume=7761328, amount=10693452836.0, id=None),
 QmtStockWeeklyOri(stock_code='000001.SZ', time=datetime.datetime(2022, 7, 24, 0, 0), open=np.float64(11.41), high=np.float64(11.63), low=np.float64(11.01), close=np.float64(11.2), volume=6057460, amount=7963091533.0, id=None),
 QmtStockWeeklyOri(stock_code='000001.SZ', time=datetime.datetime(2022, 7, 31, 0, 0), op

In [36]:
from sqlalchemy.orm import Session
from app.core.config import settings
from sqlalchemy import create_engine
from app.utils.db_utils import insert_on_duplicate_update_for_kline


In [40]:


engine = create_engine(settings.SQLALCHEMY_MYSQL_DATABASE_URI, echo=False)
with Session(engine) as db:
    affected_rows = insert_on_duplicate_update_for_kline(
        db=db,
        model_cls=QmtStockWeeklyOri,
        objs=stock_objs,
        auto_commit=True
    )
    print(f'插入或更新股票{stock_code}的行情数据成功，受影响行数：{affected_rows}')

插入或更新股票000001.SZ的行情数据成功，受影响行数：153


In [46]:
# 获取10年前的日期
ten_years_ago = (datetime.now() - timedelta(days=10 * 365)).strftime('%Y%m%d')
divid_data = xtdata.get_divid_factors("000001.SZ", start_time=three_years_ago, end_time=today)
print(divid_data)

                  time  interest  stockBonus  stockGift  allotNum  allotPrice  \
20220722  1.658419e+12     0.228         0.0        0.0       0.0         0.0   
20230614  1.686672e+12     0.285         0.0        0.0       0.0         0.0   
20240614  1.718294e+12     0.719         0.0        0.0       0.0         0.0   
20241010  1.728490e+12     0.246         0.0        0.0       0.0         0.0   
20250612  1.749658e+12     0.362         0.0        0.0       0.0         0.0   

          gugai        dr  
20220722    0.0  1.017996  
20230614    0.0  1.024369  
20240614    0.0  1.071428  
20241010    0.0  1.021872  
20250612    0.0  1.031331  
